In [ ]:
a = range(10)
a_iter = iter(a)

while True:
    try:
        b = next(a_iter)
        print(b)
    except StopIteration:
        break






0
1
2
3
4
5
6
7
8
9


In [ ]:
from typing import TypedDict, Annotated, ClassVar, TypeVar, List, Protocol
from langgraph.graph import StateGraph, START, END
from pydantic import Field, ValidationError, validate_call

# ============================================================================
# GENERIC SORT FUNCTION (Template function for different types)
# ============================================================================

# Option 1: Using Protocol to define comparable constraint
class Comparable(Protocol):
    """Protocol for types that support comparison operators."""
    def __lt__(self, other) -> bool: ...
    def __le__(self, other) -> bool: ...
    def __gt__(self, other) -> bool: ...
    def __ge__(self, other) -> bool: ...

# Define TypeVar with bound to Comparable protocol
# This restricts T to only types that implement comparison operators
T = TypeVar('T', bound=Comparable)

# Option 2: Using bound with a union of known comparable types (alternative)
from typing import Union
# T2 = TypeVar('T2', bound=Union[int, float, str])  # Less flexible

# Option 3: Using SupportsRichComparison (if available in your Python version)
# from typing import SupportsRichComparison
# T3 = TypeVar('T3', bound=SupportsRichComparison)

# Note: In Python 3.9+, you can use built-in list[] instead of List[]
# For Python 3.9+: def sort_list(items: list[T]) -> list[T]:

# Option 1: Generic sort function using TypeVar
def sort_list(items: List[T]) -> List[T]:
    """
    Generic sort function that works with any comparable type.
    
    Args:
        items: List of comparable items (int, str, float, etc.)
    
    Returns:
        Sorted list of the same type
    """
    # Create a copy to avoid modifying the original list
    result = items.copy()
    # Simple bubble sort implementation
    n = len(result)
    for i in range(n):
        for j in range(0, n - i - 1):
            if result[j] > result[j + 1]:
                result[j], result[j + 1] = result[j + 1], result[j]
        
        print(result)
    return result


# Option 2: Using built-in sorted() with type hints (simpler)
def sort_list_builtin(items: List[T]) -> List[T]:
    """Generic sort using Python's built-in sorted() function."""
    return sorted(items)


# Option 3: Generic sort with reverse parameter
def sort_list_reverse(items: List[T], reverse: bool = False) -> List[T]:
    """Generic sort with optional reverse parameter."""
    return sorted(items, reverse=reverse)


# Option 4: Generic sort with custom key function
from typing import Callable, Any
K = TypeVar('K')  # Type for the key

def sort_list_by_key(items: List[T], key: Callable[[T], K] | None = None) -> List[T]:
    """Generic sort with custom key function."""
    return sorted(items, key=key)


# Examples with different types:
print("=== Generic Sort Function Examples ===\n")

# Sort integers
int_list = [3, 18, 4, 14, 5, 9, 2, 6, 1]
print(f"Original int list: {int_list}")
print(f"Sorted: {sort_list(int_list)}")

# Sort strings
str_list = ["banana", "apple", "cherry", "date"]
print(f"\nOriginal str list: {str_list}")
print(f"Sorted: {sort_list(str_list)}")

# Sort floats
float_list = [3.14, 2.71, 1.41, 1.73]
print(f"\nOriginal float list: {float_list}")
print(f"Sorted: {sort_list(float_list)}")

# Sort with reverse
print(f"\nReverse sorted ints: {sort_list_reverse(int_list, reverse=True)}")

# Sort with custom key (e.g., by length for strings)
words = ["apple", "pie", "banana", "a"]
print(f"\nOriginal words: {words}")
print(f"Sorted by length: {sort_list_by_key(words, key=len)}")

# Sort tuples (works with any comparable type)
tuple_list = [(3, 'c'), (1, 'a'), (2, 'b')]
print(f"\nOriginal tuple list: {tuple_list}")
print(f"Sorted: {sort_list(tuple_list)}")

# ============================================================================
# DEMONSTRATING THE COMPARABLE CONSTRAINT
# ============================================================================

print("\n" + "="*70)
print("DEMONSTRATING THE COMPARABLE CONSTRAINT")
print("="*70)

# Custom class that implements comparison (works with bound=Comparable)
class Person:
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age
    
    def __lt__(self, other):
        return self.age < other.age
    
    def __le__(self, other):
        return self.age <= other.age
    
    def __gt__(self, other):
        return self.age > other.age
    
    def __ge__(self, other):
        return self.age >= other.age
    
    def __repr__(self):
        return f"Person({self.name}, {self.age})"

# This works because Person implements comparison operators
people = [Person("Alice", 30), Person("Bob", 25), Person("Charlie", 35)]
print(f"\nOriginal people: {people}")
print(f"Sorted by age: {sort_list(people)}")

# Non-comparable class (doesn't implement comparison)
class NonComparable:
    def __init__(self, value: int):
        self.value = value
    
    def __repr__(self):
        return f"NonComparable({self.value})"

# This will work at runtime but type checkers will warn
# because NonComparable doesn't satisfy the Comparable protocol
non_comparable_list = [NonComparable(3), NonComparable(1), NonComparable(2)]
print(f"\nNon-comparable list: {non_comparable_list}")
try:
    sorted_result = sort_list(non_comparable_list)
    print(f"This would fail at runtime: {sorted_result}")
except TypeError as e:
    print(f"Runtime error (expected): {e}")

print("\n--- Summary ---")
print("✅ Types that work (implement comparison): int, str, float, tuple, Person")
print("❌ Types that don't work: NonComparable (no __lt__, __gt__, etc.)")
print("💡 The 'bound=Comparable' ensures type checkers warn about non-comparable types")


In [ ]:
# ============================================================================
# DIFFERENCE BETWEEN [] AND List IN TYPE HINTS
# ============================================================================

"""
In Python type hints, there are two ways to specify a list type:

1. List[T] from typing module (Python < 3.9, or for compatibility)
2. list[T] built-in syntax (Python 3.9+)

Both are used ONLY for type hints/annotations - they don't create actual lists!
"""

from typing import List, Dict, Tuple, Set

# ============================================================================
# 1. List[T] from typing module (older style, Python < 3.9)
# ============================================================================

def process_numbers_old(items: List[int]) -> List[str]:
    """Using List from typing module."""
    return [str(x) for x in items]

# ============================================================================
# 2. list[T] built-in syntax (newer style, Python 3.9+)
# ============================================================================

def process_numbers_new(items: list[int]) -> list[str]:
    """Using built-in list[] syntax (Python 3.9+)."""
    return [str(x) for x in items]

# ============================================================================
# KEY DIFFERENCES
# ============================================================================

print("="*70)
print("DIFFERENCE BETWEEN [] AND List IN TYPE HINTS")
print("="*70)

print("\n1. SYNTAX:")
print("   - List[int]  → from typing import List (Python < 3.9)")
print("   - list[int]  → Built-in syntax (Python 3.9+)")
print("   - []         → Creates an actual empty list (runtime value)")

print("\n2. WHEN TO USE:")
print("   - List[T]    → Python < 3.9, or for backward compatibility")
print("   - list[T]    → Python 3.9+ (preferred, cleaner)")
print("   - []         → When you need an actual list object")

print("\n3. EXAMPLES:")

# Type hints (annotations only - don't create lists!)
def example1(items: List[int]) -> List[str]:
    """Type hint using List from typing."""
    return [str(x) for x in items]

def example2(items: list[int]) -> list[str]:
    """Type hint using built-in list[] syntax."""
    return [str(x) for x in items]

# Actual list creation (runtime values)
actual_list1 = []  # Empty list
actual_list2 = [1, 2, 3]  # List with values
actual_list3 = list()  # Empty list (alternative syntax)

print(f"\n   Type hint: List[int] or list[int]")
print(f"   Actual list: {actual_list1} or {actual_list2}")

print("\n4. OTHER COLLECTIONS:")
print("   - Dict[K, V]  vs  dict[K, V]")
print("   - Tuple[T]    vs  tuple[T]")
print("   - Set[T]      vs  set[T]")

# Examples
def example_dict_old(data: Dict[str, int]) -> Dict[str, str]:
    """Old style."""
    return {k: str(v) for k, v in data.items()}

def example_dict_new(data: dict[str, int]) -> dict[str, str]:
    """New style (Python 3.9+)."""
    return {k: str(v) for k, v in data.items()}

print("\n5. IMPORTANT NOTES:")
print("   ✅ Both List[T] and list[T] are ONLY for type hints")
print("   ✅ They don't affect runtime behavior")
print("   ✅ Use [] or list() to create actual lists")
print("   ✅ Python 3.9+ prefers built-in syntax (list, dict, tuple, set)")

print("\n6. PRACTICAL EXAMPLE:")

# Type hint (annotation only)
def add_one(numbers: list[int]) -> list[int]:
    """Type hint tells us: expects list of ints, returns list of ints."""
    return [x + 1 for x in numbers]

# Actual usage
my_numbers = [1, 2, 3]  # Actual list creation
result = add_one(my_numbers)
print(f"   Input:  {my_numbers}")
print(f"   Output: {result}")

print("\n" + "="*70)
print("SUMMARY: Use list[T] for type hints (Python 3.9+), use [] for actual lists")
print("="*70)


In [ ]:
# ============================================================================
# COMPARISON: PydanticAI Agent vs LangGraph Agent
# ============================================================================

"""
This cell provides a comprehensive comparison between PydanticAI Agent and 
LangGraph Agent frameworks for building AI agents.
"""

print("="*80)
print("PYDANTICAI AGENT vs LANGGRAPH AGENT - COMPREHENSIVE COMPARISON")
print("="*80)

print("\n" + "="*80)
print("1. ARCHITECTURE & PHILOSOPHY")
print("="*80)

print("\n📦 PYDANTICAI AGENT:")
print("   • Simple, function-based architecture")
print("   • Uses vanilla Python for control flow")
print("   • Agent is a single class: Agent(model, system_prompt)")
print("   • Direct function calls: agent.run_sync() or agent.run()")
print("   • Minimal abstraction layer")

print("\n🕸️  LANGGRAPH AGENT:")
print("   • Graph-based state machine architecture")
print("   • Uses StateGraph for workflow orchestration")
print("   • Nodes represent functions, edges represent transitions")
print("   • State is managed through TypedDict")
print("   • More complex but more flexible for multi-step workflows")

print("\n" + "="*80)
print("2. CODE STRUCTURE & COMPLEXITY")
print("="*80)

print("\n📦 PYDANTICAI AGENT:")
print("""
   from pydantic_ai import Agent
   
   agent = Agent(
       model='openai:gpt-4o-mini',
       system_prompt='You are a helpful assistant.',
   )
   
   result = agent.run_sync("Hello!")
   print(result.data)
""")

print("\n🕸️  LANGGRAPH AGENT:")
print("""
   from langgraph.graph import StateGraph, START, END
   from typing import TypedDict
   
   class AgentState(TypedDict):
       messages: list
       # ... other state fields
   
   def node_function(state: AgentState) -> AgentState:
       # Process state
       return state
   
   workflow = StateGraph(AgentState)
   workflow.add_node("node_name", node_function)
   workflow.add_edge(START, "node_name")
   workflow.add_edge("node_name", END)
   app = workflow.compile()
   
   result = app.invoke({"messages": [...]})
""")

print("\n" + "="*80)
print("3. STATE MANAGEMENT")
print("="*80)

print("\n📦 PYDANTICAI AGENT:")
print("   • No built-in state management")
print("   • You manually manage conversation history")
print("   • Pass message_history parameter explicitly")
print("   • Simple but requires manual implementation")
print("""
   Example:
   chat_history = get_chat_history(session_id)
   result = agent.run_sync(user_message, message_history=chat_history)
   store_messages_in_history(session_id, result)
""")

print("\n🕸️  LANGGRAPH AGENT:")
print("   • Built-in state management through TypedDict")
print("   • State persists across nodes automatically")
print("   • Each node receives and returns state")
print("   • Supports checkpointing for persistence")
print("""
   Example:
   class AgentState(TypedDict):
       messages: list
       user_query: str
       results: dict
   
   # State flows automatically between nodes
   def process_node(state: AgentState) -> AgentState:
       # state is automatically passed and updated
       return {"messages": state["messages"] + [new_message]}
""")

print("\n" + "="*80)
print("4. TYPE SAFETY & VALIDATION")
print("="*80)

print("\n📦 PYDANTICAI AGENT:")
print("   ✅ Built on Pydantic - strong type safety")
print("   ✅ Automatic validation of inputs/outputs")
print("   ✅ Structured outputs with Pydantic models")
print("   ✅ Type hints throughout")
print("   ✅ Static type checking support")

print("\n🕸️  LANGGRAPH AGENT:")
print("   ⚠️  Uses TypedDict for state (runtime type hints)")
print("   ⚠️  Less strict validation by default")
print("   ⚠️  Can add Pydantic models manually")
print("   ⚠️  Type safety depends on implementation")

print("\n" + "="*80)
print("5. WORKFLOW & CONTROL FLOW")
print("="*80)

print("\n📦 PYDANTICAI AGENT:")
print("   • Linear execution: input → agent → output")
print("   • Use Python if/else, loops for control flow")
print("   • Simple for straightforward conversations")
print("   • Can chain multiple agent calls manually")
print("""
   Example:
   result1 = agent.run_sync(query1)
   if result1.data == "need_search":
       search_result = search_tool(query1)
       result2 = agent.run_sync(query1, context=search_result)
""")

print("\n🕸️  LANGGRAPH AGENT:")
print("   • Graph-based execution with conditional edges")
print("   • Built-in support for loops and branching")
print("   • Complex multi-step workflows")
print("   • Conditional routing between nodes")
print("""
   Example:
   def should_continue(state: AgentState) -> str:
       if state["need_search"]:
           return "search_node"
       return "end"
   
   workflow.add_conditional_edges(
       "classify_node",
       should_continue,
       {"search_node": "search_node", "end": END}
   )
""")

print("\n" + "="*80)
print("6. TOOLS & FUNCTION CALLING")
print("="*80)

print("\n📦 PYDANTICAI AGENT:")
print("   • Uses dependency injection for tools")
print("   • Tools defined as functions with type hints")
print("   • Automatic tool discovery and validation")
print("""
   Example:
   @agent.tool
   def calculate(expression: str) -> float:
       return eval(expression)
   
   # Tool automatically available to agent
""")

print("\n🕸️  LANGGRAPH AGENT:")
print("   • Tools typically bound to LLM")
print("   • Tools executed in specific nodes")
print("   • More manual tool integration")
print("""
   Example:
   from langchain_core.tools import tool
   
   @tool
   def calculate(expression: str) -> float:
       return eval(expression)
   
   llm_with_tools = llm.bind_tools([calculate])
   # Use in a node
""")

print("\n" + "="*80)
print("7. USE CASES")
print("="*80)

print("\n📦 PYDANTICAI AGENT - Best for:")
print("   ✅ Simple conversational agents")
print("   ✅ Single-step or linear workflows")
print("   ✅ Type-safe, production-ready applications")
print("   ✅ Quick prototyping")
print("   ✅ When you want minimal abstraction")
print("   ✅ Applications requiring strong validation")

print("\n🕸️  LANGGRAPH AGENT - Best for:")
print("   ✅ Complex multi-step workflows")
print("   ✅ State machines and decision trees")
print("   ✅ Agents with multiple decision points")
print("   ✅ Workflows requiring loops/retries")
print("   ✅ When you need visual workflow representation")
print("   ✅ Complex agent orchestration")

print("\n" + "="*80)
print("8. LEARNING CURVE")
print("="*80)

print("\n📦 PYDANTICAI AGENT:")
print("   • ⭐⭐ Low learning curve")
print("   • Familiar Python patterns")
print("   • Minimal concepts to learn")
print("   • Quick to get started")

print("\n🕸️  LANGGRAPH AGENT:")
print("   • ⭐⭐⭐⭐ Higher learning curve")
print("   • Need to understand graph concepts")
print("   • State management patterns")
print("   • More concepts: nodes, edges, conditional routing")

print("\n" + "="*80)
print("9. DEBUGGING & OBSERVABILITY")
print("="*80)

print("\n📦 PYDANTICAI AGENT:")
print("   • Integrated with Logfire (Pydantic's observability)")
print("   • Simple debugging - standard Python debugging")
print("   • Clear error messages from Pydantic validation")

print("\n🕸️  LANGGRAPH AGENT:")
print("   • LangSmith integration for tracing")
print("   • Can visualize graph structure")
print("   • State inspection at each node")
print("   • More complex debugging due to graph nature")

print("\n" + "="*80)
print("10. PERFORMANCE & SCALABILITY")
print("="*80)

print("\n📦 PYDANTICAI AGENT:")
print("   • Lightweight - minimal overhead")
print("   • Fast for simple use cases")
print("   • Good for high-throughput simple agents")

print("\n🕸️  LANGGRAPH AGENT:")
print("   • More overhead due to graph execution")
print("   • Better for complex workflows")
print("   • Can optimize with checkpointing")
print("   • Supports parallel node execution")

print("\n" + "="*80)
print("SUMMARY TABLE")
print("="*80)

comparison_table = """
┌─────────────────────┬──────────────────────┬──────────────────────┐
│ Feature             │ PydanticAI Agent      │ LangGraph Agent      │
├─────────────────────┼──────────────────────┼──────────────────────┤
│ Complexity          │ Low                   │ High                 │
│ Learning Curve      │ Easy                  │ Moderate-Hard        │
│ State Management    │ Manual                │ Built-in             │
│ Type Safety         │ Excellent (Pydantic)  │ Good (TypedDict)     │
│ Control Flow        │ Python (if/else)      │ Graph (conditional)  │
│ Best For            │ Simple agents         │ Complex workflows    │
│ Validation          │ Automatic (Pydantic)   │ Manual               │
│ Debugging           │ Simple                │ Complex              │
│ Workflow Visual     │ No                    │ Yes (graph)          │
│ Production Ready    │ Yes                   │ Yes                  │
│ Community           │ Growing               │ Established          │
└─────────────────────┴──────────────────────┴──────────────────────┘
"""

print(comparison_table)

print("\n" + "="*80)
print("RECOMMENDATION")
print("="*80)

print("""
🎯 Choose PydanticAI Agent if:
   • You need a simple conversational agent
   • You value type safety and validation
   • You prefer vanilla Python control flow
   • You want quick development
   • Your workflow is linear or simple

🎯 Choose LangGraph Agent if:
   • You need complex multi-step workflows
   • You require state machines
   • You need conditional routing/loops
   • You want visual workflow representation
   • Your agent has multiple decision points
   • You need checkpointing/persistence

💡 Hybrid Approach:
   You can use both! PydanticAI for individual agent nodes
   and LangGraph for orchestrating multiple agents.
""")

print("\n" + "="*80)
print("EXAMPLE: Same Task in Both Frameworks")
print("="*80)

print("\n📦 PYDANTICAI - Simple Conversational Agent:")
print("""
from pydantic_ai import Agent

agent = Agent(model='openai:gpt-4o-mini', system_prompt='Helpful assistant')

# Simple conversation with history
history = []
while True:
    user_input = input("You: ")
    result = agent.run_sync(user_input, message_history=history)
    history.extend(result.all_messages())
    print(f"AI: {result.data}")
""")

print("\n🕸️  LANGGRAPH - Same Agent with Graph:")
print("""
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

class ChatState(TypedDict):
    messages: list

def chat_node(state: ChatState) -> ChatState:
    result = llm.invoke(state["messages"])
    return {"messages": state["messages"] + [result]}

workflow = StateGraph(ChatState)
workflow.add_node("chat", chat_node)
workflow.add_edge(START, "chat")
workflow.add_edge("chat", END)
app = workflow.compile()

# Usage
result = app.invoke({"messages": [HumanMessage(content="Hello")]})
""")

print("\n" + "="*80)
print("CONCLUSION")
print("="*80)

print("""
Both frameworks are excellent choices, but serve different purposes:

• PydanticAI = Simplicity + Type Safety + Quick Development
• LangGraph = Flexibility + Complex Workflows + State Management

The choice depends on your specific use case and complexity requirements.
For most simple conversational agents, PydanticAI is easier and faster.
For complex multi-agent systems or workflows, LangGraph provides more power.
""")

print("="*80)
